# Preprocessing

## resize

In [1]:
import os
import numpy as np
import json
import torch
# from google.colab import drive
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from collections import Counter
import math
from torch.nn import BCEWithLogitsLoss()

In [2]:
# drive.mount('/content/drive')
FILE_PATH =  "C:\\Users\\dipreez\\Downloads\\해양 침적 쓰레기 이미지\\Training"

In [3]:
os.chdir(FILE_PATH)

In [4]:
def resize_images(image_dir, output_dir, size):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = os.listdir(image_dir)
    num_images = len(images)
    

    for i, image in enumerate(images[:5]):
        with open(os.path.join(image_dir, image), 'r+b') as f:
            with Image.open(f) as img:
                img = img.resize(size, Image.ANTIALIAS)
                img.save(os.path.join(output_dir, image), img.format)
        if(i + 1) % 10000 == 0:
            print("[{}/{}] Image Resized".format(i + 1, num_images))

In [5]:
img_size = [64, 64]
resize_images(FILE_PATH, './data', img_size)

In [6]:
def tokenize(input_dir, output_path):
    """function for tokenizing and getting id for image

    Args: 
        input_dir: directory path of which contains images
        output_file: name of output file
    
    Returns:
        output file
    
    Notes: 
        image name format: {class name(contains space)}_{location: int}_{image no: int}.jpg

    """
    # TODO: class를 int로 변경 & 저장
    names = os.listdir(input_dir)
    explain = dict()
    full_names = dict()
    for name in names[:5]:
        tokens = name.split('.')[0]  # remove file formatter
        tokens = tokens.split('_')
        classes = []
        id=""
        for token in tokens:
            if(token.isdigit()):
                id += token
            else:
                classes.append(token)
        explain[id] = classes
        full_names[id] = name
    
    with open(output_path + "imgclass_map.json", 'w', encoding="utf-8") as f:
        json.dump(explain, f)
    with open(output_path + "imgname_map.json", 'w', encoding="utf-8") as f:
        json.dump(full_names, f)

    return explain, full_names

In [7]:
explain, full_names = tokenize(FILE_PATH, './data/')

In [8]:
print(full_names)

{'00200012': 'bundle of ropes_002_00012.jpg', '00200039': 'bundle of ropes_002_00039.jpg', '00200084': 'bundle of ropes_002_00084.jpg', '00200147': 'bundle of ropes_002_00147.jpg', '00200156': 'bundle of ropes_002_00156.jpg'}


- 클래스를 int로 변경
- 변경한 결과를 json 파일로 저장
- 추가로 그 클래스가 몇 개인지도 저장

In [9]:
class Debris(object):
    def __init__(self, debris_file, img_mapped):
        self.debris_file = debris_file
        self.img_mapped = img_mapped
        self.debris2idx = {}
        self.idx2debris=[]
        self.idx = 0
        self.debris_cnt = Counter()

        self.read_file()
    
    def read_file(self):
        if(os.path.exists(self.debris_file)):
            with open(self.debris_file, 'r') as f:
                debris = json.load(f)
                self.debris2idx = debris['debris2idx']
                self.idx2debris = debris['idx2debris']
                self.idx = len(self.idx2debris) + 1
                self.debris_cnt = Counter(debris['debris_cnt'])
        else:
            self.build_debris()
            with open(self.debris_file, 'w', encoding = 'utf-8') as f:
                json.dump({
                    'debris2idx': self.debris2idx,
                    'idx2debris': self.idx2debris,
                    'debris_cnt': self.debris_cnt
                }, f)
    
    def add_debris(self, debris):
        if not debris in self.debris2idx:
            self.debris2idx[debris] = self.idx
            self.idx2debris.append(debris)
            self.idx += 1

    def build_debris(self):
        for _, debris in self.img_mapped.items():
            for word in debris:
                self.add_debris(word)
            self.debris_cnt.update(debris)

    def __call__(self, debris):
        return self.debris2idx[debris]
    
    def __len__(self):
        return len(self.debris2idx)


In [10]:
class MarinTrashDS(Dataset):
    def __init__(self, explain, full_names, debris_file, trasform=None):
        self.debris = Debris(debris_file = debris_file, img_mapped=explain)
        self.transform = transform
        self.img_index =  [ele for ele in explain]

    def __getitem__(self, index):
        img_id = self.img_index[index]
        classes = explain[img_id]
        target = []
        for word in classes:           
            target.append(self.debris(word))
        target = torch.Tensor(target).long()

        path = './data/' + full_names[img_id]        
        img = Image.open(path).convert('RGB')

        if self.transform is not None: 
            img = self.transform(img)

        return img, target
    
    def __len__(self):
        return len(self.img_index)


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = 0.5, std = 0.5)                                
])

In [12]:
train_dataset = MarinTrashDS(explain = explain, full_names = full_names, debris_file = './data/debris.json', trasform = transform)


In [13]:
tr_index, val_index = train_test_split(list(range(len(train_dataset))), test_size = 0.2, shuffle = True)
tr_sampler = SubsetRandomSampler(tr_index)
val_sampler = SubsetRandomSampler(val_index)

In [14]:
train_loader = DataLoader(train_dataset, batch_size = 32, num_workers=0, sampler = tr_sampler)
val_loader = DataLoader(train_dataset, batch_size = 32, num_workers = 0, sampler = val_sampler)

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [22]:
from typing import Type, Any, Callable, Union, List, Optional

import torch
import torch.nn as nn
from torch import Tensor

def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
    ) -> None:
        super(BasicBlock, self).__init__()
        norm_layer = nn.BatchNorm2d
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample=downsample
        self.stride = stride


    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)


        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
    ) -> None:
        super(Bottleneck, self).__init__()
        norm_layer = nn.BatchNorm2d
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = norm_layer(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion, stride)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        ########################################
        num_classes: int = 1,
        ########################################
    ) -> None:
        super(ResNet, self).__init__()
        norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        ########################################
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        ########################################
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample=None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [23]:
net = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)

criterion = BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [24]:
train_total = len(tr_index)
valid_total = len(val_index)

train_batches = len(train_loader)
valid_batches = len(val_loader)

# Variables for lr scheduling and early stopping
best_valid_loss = 1024    # Any large number will suffice
patience = 0    # Bad epoch counter

In [26]:
# %%time
epochs =100
for epoch in range(1,epochs+1):
    # Train
    net.train()
    
    train_loss = 0
    train_correct = 0
    
    for x, y in train_loader:
        x = x.to(device)
        y = torch.tensor(y, dtype=torch.long, device=device)
        outputs = net(x)
        print(outputs.shape,y.shape)
        loss = criterion(outputs, y)
        
        optimizer.zero_grad()
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_correct += predicted.eq(y).sum().item()
        
    train_loss = train_loss / train_batches
    train_acc = train_correct / train_total
    
    # Validate
    net.eval()
    
    valid_loss = 0
    valid_correct = 0
    
    with torch.no_grad():
        for x, y in valid_loader:
            x = x.to(device)
            y = torch.tensor(y, dtype=torch.long, device=device)
            outputs = net(x)
            loss = criterion(outputs, y)
            
            valid_loss += loss.item()
            _, predicted = outputs.max(1)
            valid_correct += predicted.eq(y).sum().item()
            
    valid_loss = valid_loss / valid_batches
    valid_acc = valid_correct / valid_total
    
    # Save best model
    if best_valid_loss > valid_loss:
        torch.save(net.state_dict(), './2018310773_임승재.pt')
        best_valid_loss = valid_loss
        patience = 0

    if epoch%(epochs/10) == 0:
      print('[%d/%d] TrainLoss: %.3f, ValLoss: %.3f | TrainAcc: %.2f, ValAcc: %.2f' % (epoch, epochs, train_loss, valid_loss, train_acc, valid_acc))
    
    scheduler.step()
    
    # Break training loop if no improvement for 10 consecutive epochs
    if patience == 10:
        print('No improvement found. Escaped for loop at [%d/%d]' % (epoch,epochs))
        break
        
    patience += 1

torch.Size([4, 1]) torch.Size([4, 1])


C:\Users\dipreez\AppData\Local\Temp/ipykernel_32792/4236042798.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.long, device=device)


RuntimeError: multi-target not supported at C:/cb/pytorch_1000000000000/work/aten/src\THCUNN/generic/ClassNLLCriterion.cu:15